<a href="https://colab.research.google.com/github/Vasanthkumarredd/ecommerece/blob/main/ml_project_batch16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets torch torchvision transformers matplotlib
!pip install -q pytorch-lightning
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18



In [ ]:
tqdm.pandas()

# Load dataset from Hugging Face
dataset = load_dataset("c3rl/IIIT-INDIC-HW-WORDS-Hindi")
train_ds = dataset['train']
val_ds = dataset['validation']
test_ds = dataset['test']

NameError: name 'tqdm' is not defined

In [ ]:
from pathlib import Path

# Create directory if needed
Path("Data/Characters").mkdir(parents=True, exist_ok=True)

# Check if character CSV exists, create if missing
character_df_fp = 'Data/Characters/Characters-Hindi.csv'
if not Path(character_df_fp).exists():
    # Generate character CSV from dataset
    def extract_characters(dataset_split):
        chars = set()
        for item in tqdm(dataset_split, desc="Extracting characters"):
            text = item['text']
            chars.update(list(text))
        return chars

    train_chars = extract_characters(train_ds)
    val_chars = extract_characters(val_ds)
    test_chars = extract_characters(test_ds)
    all_chars = train_chars.union(val_chars).union(test_chars)
    characters = ["-"] + sorted(list(all_chars))

    pd.DataFrame({"Character": characters}).to_csv(character_df_fp, index=False)

# Load character DataFrame
character_df = pd.read_csv(character_df_fp)
print(f"Loaded character mapping with {len(character_df)} characters")
print(character_df.head())

# Rest of your existing character processing code...
characters = ["-"] + sorted(list(all_chars))  # Remove this line if exists

character_df = pd.DataFrame({"Character": characters})
character_df.to_csv("Data/Characters/Characters-Hindi.csv", index=False)
print("Generated Characters-Hindi.csv with", len(character_df), "characters")

Extracting characters:   0%|          | 0/69853 [00:00<?, ?it/s]

Extracting characters:   0%|          | 0/12708 [00:00<?, ?it/s]

Extracting characters:   0%|          | 0/12869 [00:00<?, ?it/s]

Loaded character mapping with 109 characters
  Character
0         -
1         -
2         ँ
3         ं
4         ः
Generated Characters-Hindi.csv with 109 characters


In [ ]:
import os

class HuggingFaceHindiDataset(Dataset):
    def __init__(self, dataset_split):
        self.dataset = dataset_split
        self.transform = transforms.Compose([
            transforms.Resize((32, 128)),  # Resize images to a fixed size
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        item = self.dataset[index]
        image = item['image'].convert('RGB')
        text = item['text']
        image = self.transform(image)
        return image, text, ""  # Empty string for compatibility

# Create datasets and dataloaders
trainset = HuggingFaceHindiDataset(train_ds)
valset = HuggingFaceHindiDataset(val_ds)
testset = HuggingFaceHindiDataset(test_ds)

batch_size = 64
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())

In [ ]:
image_batch, text_batch, image_fn_batch = next(iter(train_loader))
print(image_batch.size())
print(text_batch)

torch.Size([64, 3, 32, 128])
('आश्रितों', 'मरीचिका', 'शासन', 'सबूरी', 'विधा', 'मारा।', 'ट्वीट', 'जायका', 'मेथवाड़ा', 'यह', 'लाठियों', 'जागते', 'जुड़', 'चित्रकार', 'झुका', 'मशीन-औजार', 'फाके', 'पकड़-पकड़', 'संकर', 'छोड़ते', 'सोचकर', 'धृतराष्ट्र', 'बेहूदी', 'भाइयो', 'पहनकर', 'बोलेंगे', 'बुलेटप्रूफ', 'सौपेंगे', 'गुर्राती', 'ग़ायब', 'जनसुरक्षा', 'खुलेआम', 'बुक', 'जा', 'निमित्त', 'अत्यधिक', 'वंश', 'भाभा', 'रोगों', 'नहीं-नहीं', 'फ़ूल', 'संगीता', 'सरगना', 'बिजी', 'साठगांठ', 'यूरोपीय', 'अनाथ', 'स्नानगृह', 'उपायुक्त', 'फायदा', 'आराम', 'राजभाषा', 'कपडा', 'हीं', 'टीचर', 'रिकवरी', 'ग्रंथि', 'कामज़ात', 'फाँसीघर', 'बढाना', 'जातीं', 'अपाइंटमैंट', 'लाने-ले', 'नीदं')


In [ ]:
def tokenize(text):

    text_tokens = list(text)

    return text_tokens

In [ ]:
for text in text_batch:
    print(text, tokenize(text))

आश्रितों ['आ', 'श', '्', 'र', 'ि', 'त', 'ो', 'ं']
मरीचिका ['म', 'र', 'ी', 'च', 'ि', 'क', 'ा']
शासन ['श', 'ा', 'स', 'न']
सबूरी ['स', 'ब', 'ू', 'र', 'ी']
विधा ['व', 'ि', 'ध', 'ा']
मारा। ['म', 'ा', 'र', 'ा', '।']
ट्वीट ['ट', '्', 'व', 'ी', 'ट']
जायका ['ज', 'ा', 'य', 'क', 'ा']
मेथवाड़ा ['म', 'े', 'थ', 'व', 'ा', 'ड़', 'ा']
यह ['य', 'ह']
लाठियों ['ल', 'ा', 'ठ', 'ि', 'य', 'ो', 'ं']
जागते ['ज', 'ा', 'ग', 'त', 'े']
जुड़ ['ज', 'ु', 'ड़']
चित्रकार ['च', 'ि', 'त', '्', 'र', 'क', 'ा', 'र']
झुका ['झ', 'ु', 'क', 'ा']
मशीन-औजार ['म', 'श', 'ी', 'न', '-', 'औ', 'ज', 'ा', 'र']
फाके ['फ', 'ा', 'क', 'े']
पकड़-पकड़ ['प', 'क', 'ड', '़', '-', 'प', 'क', 'ड', '़']
संकर ['स', 'ं', 'क', 'र']
छोड़ते ['छ', 'ो', 'ड़', 'त', 'े']
सोचकर ['स', 'ो', 'च', 'क', 'र']
धृतराष्ट्र ['ध', 'ृ', 'त', 'र', 'ा', 'ष', '्', 'ट', '्', 'र']
बेहूदी ['ब', 'े', 'ह', 'ू', 'द', 'ी']
भाइयो ['भ', 'ा', 'इ', 'य', 'ो']
पहनकर ['प', 'ह', 'न', 'क', 'र']
बोलेंगे ['ब', 'ो', 'ल', 'े', 'ं', 'ग', 'े']
बुलेटप्रूफ ['ब', 'ु', 'ल', 'े', 'ट', 'प', '्', 'र', 'ू', 'फ'

In [ ]:
resnet = resnet18(pretrained=True)
#print(resnet)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 179MB/s]


In [ ]:
for p in resnet.parameters():
    pass#p.requires_grad = False

In [ ]:
class CRNN(nn.Module):

    def __init__(self, num_chars, rnn_hidden_size=256):

        super(CRNN, self).__init__()
        self.num_chars = num_chars
        self.rnn_hidden_size = rnn_hidden_size
        self.dp1 = nn.Dropout(p=0.5)
        self.dp2 = nn.Dropout(p=0.5)
        self.dp3 = nn.Dropout(p=0.5)

        # CNN Part 1
        resnet_modules = list(resnet.children())[:-3]
        self.cnn_p1 = nn.Sequential(*resnet_modules)

        # CNN Part 2
        self.cnn_p2 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.linear1 = nn.Linear(512, 256)

        # RNN
        self.rnn1 = nn.GRU(input_size=rnn_hidden_size,
                            hidden_size=rnn_hidden_size,
                            bidirectional=True,
                            num_layers=2,
                            batch_first=True)
        self.rnn2 = nn.GRU(input_size=rnn_hidden_size,
                            hidden_size=rnn_hidden_size,
                            bidirectional=True,
                            num_layers=1,
                            batch_first=True)
        self.linear2 = nn.Linear(self.rnn_hidden_size*2, num_chars)


    def forward(self, batch):

        batch = self.cnn_p1(batch)
        # print(batch.size()) # torch.Size([-1, 256, 4, 13])

        batch = self.cnn_p2(batch) # [batch_size, channels, height, width]
        batch = self.dp1(batch)
        # print(batch.size())# torch.Size([-1, 256, 4, 13])

        batch = batch.permute(0, 3, 1, 2) # [batch_size, width, channels, height]
        # print(batch.size()) # torch.Size([-1, 13, 256, 4])

        batch_size = batch.size(0)
        T = batch.size(1)
        batch = batch.view(batch_size, T, -1) # [batch_size, T==width, num_features==channels*height]
        # print(batch.size()) # torch.Size([-1, 13, 1024])

        batch = self.linear1(batch)
        batch = self.dp2(batch)
        # print(batch.size()) # torch.Size([-1, 13, 256])

        batch, hidden = self.rnn1(batch)
        feature_size = batch.size(2)
        batch = batch[:, :, :feature_size//2] + batch[:, :, feature_size//2:]
        # print(batch.size()) # torch.Size([-1, 13, 256])

        batch, hidden = self.rnn2(batch)
        batch = self.dp3(batch)
        # print(batch.size()) # torch.Size([-1, 13, 512])

        batch = self.linear2(batch)
        # print(batch.size()) # torch.Size([-1, 13, 375])

        batch = batch.permute(1, 0, 2) # [T, batch_size, num_classes==num_features]
        # print(batch.size()) # torch.Size([13, -1, 375])

        return batch

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if type(m) in [nn.Linear, nn.Conv2d, nn.Conv1d]:
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

char2idx = {char: idx for idx, char in enumerate(character_df['Character'])}

# Now you can instantiate the CRNN model:
crnn = CRNN(num_chars=len(char2idx), rnn_hidden_size=256)
#crnn.apply(weights_init)
crnn = crnn.to(device)

In [ ]:
text_batch_logits = crnn(image_batch.to(device))
print(text_batch)
print(text_batch_logits.shape)

('आश्रितों', 'मरीचिका', 'शासन', 'सबूरी', 'विधा', 'मारा।', 'ट्वीट', 'जायका', 'मेथवाड़ा', 'यह', 'लाठियों', 'जागते', 'जुड़', 'चित्रकार', 'झुका', 'मशीन-औजार', 'फाके', 'पकड़-पकड़', 'संकर', 'छोड़ते', 'सोचकर', 'धृतराष्ट्र', 'बेहूदी', 'भाइयो', 'पहनकर', 'बोलेंगे', 'बुलेटप्रूफ', 'सौपेंगे', 'गुर्राती', 'ग़ायब', 'जनसुरक्षा', 'खुलेआम', 'बुक', 'जा', 'निमित्त', 'अत्यधिक', 'वंश', 'भाभा', 'रोगों', 'नहीं-नहीं', 'फ़ूल', 'संगीता', 'सरगना', 'बिजी', 'साठगांठ', 'यूरोपीय', 'अनाथ', 'स्नानगृह', 'उपायुक्त', 'फायदा', 'आराम', 'राजभाषा', 'कपडा', 'हीं', 'टीचर', 'रिकवरी', 'ग्रंथि', 'कामज़ात', 'फाँसीघर', 'बढाना', 'जातीं', 'अपाइंटमैंट', 'लाने-ले', 'नीदं')
torch.Size([8, 64, 108])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CRNN(num_chars=len(characters)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CTCLoss(blank=0)

def encode_text_batch(text_batch):
    encoded = []
    lens = []
    for text in text_batch:
        text_encoded = [char2idx[c] for c in text]
        encoded.extend(text_encoded)
        lens.append(len(text))
    return (torch.IntTensor(encoded),
            torch.IntTensor(lens))

In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 570.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # For detailed CUDA error reporting
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":16:8"  # For deterministic behavior

In [ ]:

torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

# Set environment variables for CUDA debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":16:8"
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Define device explicitly as cuda:0
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Ensure CUDA is initialized
if device.type == 'cuda':
    torch.cuda.init()
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory allocated: {torch.cuda.memory_allocated(device)/1e6:.2f} MB")

# [Assume character_df, char2idx, trainset, valset, model, optimizer, criterion are defined as in the original code]

# Training loop
num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    torch.cuda.synchronize(device=device)

    for images, texts, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        print(f"Images device before move: {images.device}")  # Debug device
        images = images.contiguous().to(device, non_blocking=False)
        print(f"Images device after move: {images.device}")  # Debug device
        assert images.device == device, f"Input tensor device mismatch! Expected {device}, got {images.device}"

        optimizer.zero_grad(set_to_none=True)
        outputs = model(images)
        log_probs = F.log_softmax(outputs, dim=2)

        input_lengths = torch.full(
            (images.size(0),),
            outputs.size(0),
            dtype=torch.int32,
            device=device
        )

        targets, target_lengths = encode_text_batch(texts)
        if targets is None or target_lengths is None:
            continue

        loss = criterion(log_probs, targets, input_lengths, target_lengths)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader) if len(train_loader) > 0 else float('inf')

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, texts, _ in tqdm(val_loader, desc="Validation"):
            images = images.contiguous().to(device, non_blocking=True)
            assert images.device == device, f"Validation input tensor device mismatch! Expected {device}, got {images.device}"
            outputs = model(images)
            log_probs = F.log_softmax(outputs, dim=2)

            input_lengths = torch.full(
                (images.size(0),),
                outputs.size(0),
                dtype=torch.int32,
                device=device
            )

            targets, target_lengths = encode_text_batch(texts)
            if targets is None or target_lengths is None:
                continue

            loss = criterion(log_probs, targets, input_lengths, target_lengths)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader) if len(val_loader) > 0 else float('inf')

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"Saved best model with validation loss: {best_val_loss:.4f}")

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {avg_train_loss:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}\n")

Using device: cuda:0
CUDA device: Tesla T4
CUDA memory allocated: 188.21 MB


Epoch 1/10:   0%|          | 0/1092 [00:00<?, ?it/s]

Images device before move: cpu
Images device after move: cuda:0


Epoch 1/10:   0%|          | 1/1092 [00:03<55:56,  3.08s/it]

Images device before move: cpu
Images device after move: cuda:0
Images device before move: cpu
Images device after move: cuda:0


...

Images device before move: cpu
Images device after move: cuda:0
Images device before move: cpu
Images device after move: cuda:0


Epoch 10/10: 100%|█████████▉| 1091/1092 [03:47<00:00,  4.10it/s]

Images device before move: cpu
Images device after move: cuda:0
Images device before move: cpu
Images device after move: cuda:0


Validation: 100%|██████████| 199/199 [00:33<00:00,  5.88it/s]

Epoch 10/10
Train Loss: nan
Validation Loss: nan



In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

# Ensure device and model are set up (from your previous cells)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.eval()

# Decode function to convert model predictions to text (corrected)
def decode_predictions(log_probs, char2idx, idx2char):
    _, preds = torch.max(log_probs, dim=2)
    preds = preds.permute(1, 0)  # [T, batch_size]
    decoded_texts = []
    blank_idx = 0  # Assuming 0 is the blank token as per your CTC setup
    for i in range(preds.size(1)):
        pred = preds[:, i].tolist()
        decoded_text = []
        for j in range(len(pred)):
            if j > 0 and pred[j] == pred[j-1] and pred[j] != blank_idx:
                continue
            if pred[j] != blank_idx:  # Skip blank token
                decoded_text.append(idx2char.get(pred[j], ''))  # Use .get() to handle missing keys
        decoded_texts.append(''.join(decoded_text))
    return decoded_texts

# Evaluate accuracy on validation set
all_preds = []
all_targets = []
with torch.no_grad():
    for images, texts, _ in tqdm(val_loader, desc="Evaluating"):
        images = images.contiguous().to(device, non_blocking=True)
        outputs = model(images)
        log_probs = F.log_softmax(outputs, dim=2)
        # Create idx2char with all indices mapped, including blank
        idx2char = {v: k for k, v in char2idx.items()}  # Reverse mapping
        idx2char[0] = ''  # Map blank index to empty string (or skip it in the if condition)
        preds = decode_predictions(log_probs, char2idx, idx2char)
        all_preds.extend(preds)
        all_targets.extend(texts)

# Compute accuracy
correct = sum(1 for pred, target in zip(all_preds, all_targets) if pred == target)
accuracy = correct / len(all_targets)
print(f"Validation Accuracy: {accuracy:.4f}")

Evaluating: 100%|██████████| 199/199 [00:32<00:00,  6.07it/s]

Validation Accuracy: 0.0000


In [ ]:
!zip -r all_folders.zip /content/*
from google.colab import files
files.download('all_folders.zip')

  adding: content/Data/ (stored 0%)
  adding: content/Data/Characters/ (stored 0%)
  adding: content/Data/Characters/Characters-Hindi.csv (deflated 55%)
  adding: content/sample_data/ (stored 0%)
  adding: content/sample_data/README.md (deflated 39%)
  adding: content/sample_data/anscombe.json (deflated 83%)
  adding: content/sample_data/california_housing_train.csv (deflated 79%)
  adding: content/sample_data/california_housing_test.csv (deflated 76%)
  adding: content/sample_data/mnist_test.csv (deflated 88%)
  adding: content/sample_data/mnist_train_small.csv (deflated 88%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!git clone https://github.com/gokulkarthik/Indian-Scene-Text-Recognition.git

Cloning into 'Indian-Scene-Text-Recognition'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 82 (delta 11), reused 0 (delta 0), pack-reused 51 (from 1)
Receiving objects: 100% (82/82), 493.30 MiB | 33.53 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (44/44), done.


In [ ]:



import os
from torchvision.models import resnet18, ResNet18_Weights
resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18

tqdm.pandas()


language = 'Hindi'
data_dir = f'Data/{language}'
train_dir = f'{data_dir}/Train'
val_dir = f'{data_dir}/Val'
test_dir = f'{data_dir}/Test'

# %% [code]
character_df_fp = f'Data/Characters/Characters-{language}.csv'


# %% [code]
cpu_count = os.cpu_count()
print(cpu_count)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

character_df = pd.read_csv(character_df_fp)
print(character_df.shape)

# %% [code]
characters = set()
for character in character_df['Character'].values:
    characters.update(list(character))
characters = ["-"] + sorted(list(characters))
print(len(characters))
print(characters)



idx2char = {k:v for k,v in enumerate(characters)}
print(idx2char)
char2idx = {v:k for k,v in idx2char.items()}
print(char2idx)



class IndianSceneTextDataset(Dataset):

    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.image_fns = os.listdir(data_dir)

    def __len__(self):
        return len(self.image_fns)

    def __getitem__(self, index):
        image_fn = self.image_fns[index]
        image_fp = os.path.join(self.data_dir, image_fn)
        image = Image.open(image_fp).convert('RGB')
        image = self.transform(image)
        text = image_fn.split("_")[0][:13]
        return image, text, image_fn

    def transform(self, image):

        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)

trainset = IndianSceneTextDataset(train_dir)
valset = IndianSceneTextDataset(val_dir)
testset = IndianSceneTextDataset(test_dir)
print(len(trainset), len(valset), len(testset))

train_loader = DataLoader(trainset, batch_size=16, num_workers=os.cpu_count(), shuffle=True)
val_loader = DataLoader(valset, batch_size=16, num_workers=os.cpu_count(), shuffle=False)
test_loader = DataLoader(testset, batch_size=16, num_workers=os.cpu_count(), shuffle=False)
print(len(train_loader), len(val_loader), len(test_loader))

image_batch, text_batch, image_fn_batch = iter(train_loader).next()
print(image_batch.size())
print(text_batch)

def tokenize(text):

    text_tokens = list(text)

    return text_tokens


for text in text_batch:
    print(text, tokenize(text))

resnet = resnet18(pretrained=True)
#print(resnet)

for p in resnet.parameters():
    pass#p.requires_grad = False


class CRNN(nn.Module):

    def __init__(self, num_chars, rnn_hidden_size=256):

        super(CRNN, self).__init__()
        self.num_chars = num_chars
        self.rnn_hidden_size = rnn_hidden_size
        self.dp1 = nn.Dropout(p=0.5)
        self.dp2 = nn.Dropout(p=0.5)
        self.dp3 = nn.Dropout(p=0.5)

        # CNN Part 1
        resnet_modules = list(resnet.children())[:-3]
        self.cnn_p1 = nn.Sequential(*resnet_modules)

        # CNN Part 2
        self.cnn_p2 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.linear1 = nn.Linear(1024, 256)

        # RNN
        self.rnn1 = nn.GRU(input_size=rnn_hidden_size,
                            hidden_size=rnn_hidden_size,
                            bidirectional=True,
                            num_layers=2,
                            batch_first=True)
        self.rnn2 = nn.GRU(input_size=rnn_hidden_size,
                            hidden_size=rnn_hidden_size,
                            bidirectional=True,
                            num_layers=1,
                            batch_first=True)
        self.linear2 = nn.Linear(self.rnn_hidden_size*2, num_chars)


    def forward(self, batch):

        batch = self.cnn_p1(batch)
        # print(batch.size()) # torch.Size([-1, 256, 4, 13])

        batch = self.cnn_p2(batch) # [batch_size, channels, height, width]
        batch = self.dp1(batch)
        # print(batch.size())# torch.Size([-1, 256, 4, 13])

        batch = batch.permute(0, 3, 1, 2) # [batch_size, width, channels, height]
        # print(batch.size()) # torch.Size([-1, 13, 256, 4])

        batch_size = batch.size(0)
        T = batch.size(1)
        batch = batch.view(batch_size, T, -1) # [batch_size, T==width, num_features==channels*height]
        # print(batch.size()) # torch.Size([-1, 13, 1024])

        batch = self.linear1(batch)
        batch = self.dp2(batch)
        # print(batch.size()) # torch.Size([-1, 13, 256])

        batch, hidden = self.rnn1(batch)
        feature_size = batch.size(2)
        batch = batch[:, :, :feature_size//2] + batch[:, :, feature_size//2:]
        # print(batch.size()) # torch.Size([-1, 13, 256])

        batch, hidden = self.rnn2(batch)
        batch = self.dp3(batch)
        # print(batch.size()) # torch.Size([-1, 13, 512])

        batch = self.linear2(batch)
        # print(batch.size()) # torch.Size([-1, 13, 375])

        batch = batch.permute(1, 0, 2) # [T, batch_size, num_classes==num_features]
        # print(batch.size()) # torch.Size([13, -1, 375])

        return batch

def weights_init(m):
    classname = m.__class__.__name__
    if type(m) in [nn.Linear, nn.Conv2d, nn.Conv1d]:
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


crnn = CRNN(num_chars=len(char2idx), rnn_hidden_size=256)
#crnn.apply(weights_init)
crnn = crnn.to(device)


#crnn

text_batch_logits = crnn(image_batch.to(device))
print(text_batch)
print(text_batch_logits.shape)






criterion = nn.CTCLoss(blank=0)

def encode_text_batch(text_batch):

    text_batch = [tokenize(text) for text in text_batch]
    text_batch_targets_lens = [len(text) for text in text_batch]
    text_batch_targets_lens = torch.IntTensor(text_batch_targets_lens)

    text_batch_concat = []
    for text in text_batch:
        for token in text:
            text_batch_concat.append(token)

    text_batch_targets = [char2idx[c] for c in text_batch_concat]
    text_batch_targets = torch.IntTensor(text_batch_targets)

    return text_batch_targets, text_batch_targets_lens


def compute_loss(text_batch, text_batch_logits):
    """
    text_batch: list of strings of length equal to batch size
    text_batch_logits: Tensor of size([T, batch_size, num_classes])
    """
    text_batch_logps = F.log_softmax(text_batch_logits, 2) # [T, batch_size, num_classes]
    text_batch_logps_lens = torch.full(size=(text_batch_logps.size(1),),
                                       fill_value=text_batch_logps.size(0),
                                       dtype=torch.int32).to(device) # [batch_size]
    #print(text_batch_logps.shape)
    #print(text_batch_logps_lens)
    text_batch_targets, text_batch_targets_lens = encode_text_batch(text_batch)
    #print(text_batch_targets)
    #print(text_batch_targets_lens)
    loss = criterion(text_batch_logps, text_batch_targets, text_batch_logps_lens, text_batch_targets_lens)

    return loss


compute_loss(text_batch, text_batch_logits)


# Hyperparameters: Data Loading
batch_size = 32

# Hyperparameters: Model Architecture
rnn_hidden_size = 256

# Hyperparameters: Training
num_epochs = 2
lr = 0.001
weight_decay = 1e-5
clip_norm = 5
step_size = 5
gamma = 0.95

trainset = IndianSceneTextDataset(train_dir)
valset = IndianSceneTextDataset(val_dir)

train_loader = DataLoader(trainset, batch_size=batch_size, num_workers=os.cpu_count(), shuffle=True)
val_loader = DataLoader(valset, batch_size=batch_size, num_workers=os.cpu_count(), shuffle=False)

crnn = CRNN(num_chars=len(char2idx), rnn_hidden_size=rnn_hidden_size)
#crnn.apply(weights_init)
crnn = crnn.to(device)

optimizer = optim.Adam(crnn.parameters(), lr=lr, weight_decay=weight_decay)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)


epoch_losses = []
epoch_losses_val = []
iteration_losses = []
num_updates_epochs = []
for epoch in tqdm(range(1, num_epochs+1)):
    epoch_loss_list = []
    epoch_loss_list_val = []
    num_updates_epoch = 0

    crnn.train()
    for image_batch, text_batch, image_fn_batch in tqdm(train_loader, leave=False, desc="Train"):
        optimizer.zero_grad()
        text_batch_logits = crnn(image_batch.to(device))
        loss = compute_loss(text_batch, text_batch_logits)
        iteration_loss = loss.item()
        if np.isnan(iteration_loss) or np.isinf(iteration_loss):
            continue

        num_updates_epoch += 1
        iteration_losses.append(iteration_loss)
        epoch_loss_list.append(iteration_loss)
        loss.backward()
        nn.utils.clip_grad_norm_(crnn.parameters(), clip_norm)
        optimizer.step()

    crnn.eval()
    for image_batch, text_batch, image_fn_batch in tqdm(val_loader, leave=False, desc="Val"):
        text_batch_logits = crnn(image_batch.to(device))
        loss = compute_loss(text_batch, text_batch_logits)
        iteration_loss = loss.item()
        if np.isnan(iteration_loss) or np.isinf(iteration_loss):
            continue
        epoch_loss_list_val.append(iteration_loss)


    epoch_loss = np.mean(epoch_loss_list)
    epoch_loss_val = np.mean(epoch_loss_list_val)
    print("Epoch:{}    TrainLoss:{}    ValLoss:{}    NumUpdates:{}    LR:{}".\
          format(epoch, epoch_loss, epoch_loss_val, num_updates_epoch, optimizer.param_groups[0]['lr']))
    epoch_losses.append(epoch_loss)
    epoch_losses_val.append(epoch_loss_val)
    num_updates_epochs.append(num_updates_epoch)
    lr_scheduler.step()

    if epoch % 10 == 0:
        model_path = f'Models/{language}-Unicode-e{epoch}.pth'
        torch.save(crnn.state_dict(), model_path)


model_path = f'Models/{language}-Unicode.pth'
torch.save(crnn.state_dict(), model_path)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.plot(iteration_losses)
ax1.set_xlabel("Iterations")
ax1.set_ylabel("Loss")

ax2.plot(epoch_losses, label="Train Loss")
ax2.plot(epoch_losses_val, label="Val Loss")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")

plt.legend()
plt.show()




#trainset = IndianSceneTextDataset(train_dir)
#valset = IndianSceneTextDataset(val_dir)
testset = IndianSceneTextDataset(test_dir)

#train_loader = DataLoader(trainset, batch_size=batch_size, num_workers=os.cpu_count(), shuffle=False)
#val_loader = DataLoader(valset, batch_size=batch_size, num_workers=os.cpu_count(), shuffle=False)
test_loader = DataLoader(testset, batch_size=batch_size, num_workers=os.cpu_count(), shuffle=False)


model_path = f'Models/{language}-Unicode.pth'

crnn = CRNN(len(char2idx), rnn_hidden_size=rnn_hidden_size)
crnn.load_state_dict(torch.load(model_path))
crnn = crnn.to(device)


def decode_predictions(text_batch_logits):

    text_batch_tokens = F.softmax(text_batch_logits, 2).argmax(2) # [T, batch_size]
    text_batch_tokens = text_batch_tokens.numpy().T # [batch_size, T]

    text_batch_tokens_new = []
    for text_tokens in text_batch_tokens:
        text = [idx2char[idx] for idx in text_tokens]
        text = "".join(text)
        text_batch_tokens_new.append(text)

    return text_batch_tokens_new


def make_predictions(data_loader):
    result = pd.DataFrame(columns=['image_fn', 'actual', 'prediction'])
    with torch.no_grad():
        for image_batch, text_batch, image_fn_batch in tqdm(data_loader, leave=False):
            text_batch_logits = crnn(image_batch.to(device)) # [T, batch_size, num_classes==num_features]
            text_batch_pred = decode_predictions(text_batch_logits.cpu())
            #print(text_batch, text_batch_pred)
            df = pd.DataFrame(columns=['actual', 'prediction'])
            df['image_fn'] = image_fn_batch
            df['actual'] = text_batch
            df['prediction'] = text_batch_pred
            result = pd.concat([result, df])
    result = result.reset_index(drop=True)
    return result


# %% [code]
train_result = make_predictions(train_loader)
val_result = make_predictions(val_loader)
test_result = make_predictions(test_loader)



print(train_result.shape)
print(val_result.shape)
print(test_result.shape)
train_result.head()

def remove_duplicates(text):
    if len(text) > 1:
        letters = [text[0]] + [letter for idx, letter in enumerate(text[1:], start=1) if text[idx] != text[idx-1]]
    elif len(text) == 1:
        letters = [text[0]]
    else:
        return ""
    return "".join(letters)

def correct_prediction(word):
    parts = word.split("-")
    parts = [remove_duplicates(part) for part in parts]
    corrected_word = "".join(parts)
    return corrected_word



train_result['prediction_corrected'] = train_result['prediction'].apply(correct_prediction)
val_result['prediction_corrected'] = val_result['prediction'].apply(correct_prediction)
test_result['prediction_corrected'] = test_result['prediction'].apply(correct_prediction)



train_result['actual_length'] = train_result['actual'].apply(len)
val_result['actual_length'] = val_result['actual'].apply(len)
test_result['actual_length'] = test_result['actual'].apply(len)

train_result['prediction_length'] = train_result['prediction_corrected'].apply(len)
val_result['prediction_length'] = val_result['prediction_corrected'].apply(len)
test_result['prediction_length'] = test_result['prediction_corrected'].apply(len)


def levenshteinDistance(row):

    s1 = row['actual']
    s2 = row['prediction_corrected']
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]


train_result['edit_distance'] = train_result.progress_apply(levenshteinDistance, axis=1)
val_result['edit_distance'] = val_result.apply(levenshteinDistance, axis=1)
test_result['edit_distance'] = test_result.apply(levenshteinDistance, axis=1)



train_result.actual_length.quantile([0.9, 0.99, 0.999])



train_accuracy = accuracy_score(train_result['actual'], train_result['prediction_corrected'])
val_accuracy = accuracy_score(val_result['actual'], val_result['prediction_corrected'])
test_accuracy = accuracy_score(test_result['actual'], test_result['prediction_corrected'])
print(train_accuracy, val_accuracy, test_accuracy)


# %% [code]
for max_edit_distance in range(3+1):
    print("Max Edit Distance", max_edit_distance)
    train_accuracy = (train_result['edit_distance'] <= max_edit_distance).sum() / len(train_result)
    val_accuracy = (val_result['edit_distance'] <= max_edit_distance).sum() / len(val_result)
    test_accuracy = (test_result['edit_distance'] <= max_edit_distance).sum() / len(test_result)
    print(train_accuracy, val_accuracy, test_accuracy)
    print("*"*50)


# %% [code]
def visualize_misclassifications(split='Train', num_samples=20, max_edit_distance=2):

    if split == 'Train':
        result = train_result
    elif split == 'Val':
        result = val_result
    elif split == 'Test':
        result = test_result

    mask = result['edit_distance'] >= max_edit_distance
    result = result[mask].sample(n = num_samples)
    for row_id, row in result.iterrows():
        info = 'Actual:', row['actual'], 'Prediction:', row['prediction'], \
              'Prediction Corrected:', row['prediction_corrected']
        print(info)

        image_fp = f'Data/{language}/{split}/{row["image_fn"]}'
        plt.imshow(Image.open(image_fp))
        plt.axis(False)
        plt.show()





2
cuda


FileNotFoundError: [Errno 2] No such file or directory: 'Data/Characters/Characters-Hindi.csv'